In [1]:
import numpy as np
import pandas as pd
from sklearn import metrics, manifold
from tqdm import tqdm
import gensim.downloader as gensim_api
import transformers
import os

In [2]:
df=pd.read_csv('output1.csv',nrows=4000)
print(df["clean"])

0        rt mleew boy dats cold tyga dwn bad for cuffi...
1        rt urkindofbrand dawg rt sbaby life you ever ...
2        rt c g anderson viva based she look like a tr...
3        rt shenikaroberts the shit you hear about me ...
4        t madison x the shit just blows me claim you ...
                              ...                        
3995            msoltan koredeb the witches took them hoe
3996     msoltan matt simonson koredeb nvm thought we ...
3997     mtbornnard it s got good mojo for sure paybac...
3998     m a larson stevestreza you re saying zebras a...
3999                              m carbonaro nice pussy 
Name: clean, Length: 4000, dtype: object


In [3]:
nlp = gensim_api.load("glove-wiki-gigaword-300")

In [4]:
nlp.most_similar(["nigger"], topn=3)

[('slur', 0.5023007392883301),
 ('niggers', 0.5003143548965454),
 ('faggot', 0.4708784222602844)]

In [5]:
def get_similar_words(lst_words, top, nlp):
    lst_out = lst_words
    for tupla in nlp.most_similar(lst_words, topn=top):
        lst_out.append(tupla[0])
    return list(set(lst_out))

In [6]:
def similarWords(words, top, nlp):
    out = words
    for tupl in nlp.most_similar(words, topn=top):
        out.append(tupl[0])
    return list(set(out))

In [7]:
clusters={}
clusters['HATE']=get_similar_words(['hate'], top=30, nlp=nlp)

In [8]:
tokenizer = transformers.DistilBertTokenizer.from_pretrained('al', do_lower_case=True)
nlp = transformers.TFDistilBertModel.from_pretrained('al')

C:\Users\Rachit\anaconda3\lib\site-packages\transformers\configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some layers from the model checkpoint at al were not used when initializing TFDistilBertModel: ['vocab_layer_norm', 'vocab_transform', 'activation_13', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequ

In [9]:
def utils_bert_embedding(txt, tokenizer, nlp):
    '''
    Word embedding with Bert (equivalent to nlp["word"]).
    :parameter
        :param txt: string 
        :param tokenizer: transformers tokenizer
        :param nlp: transformers bert
    :return
        tensor sentences x words x vector (1x3x768) 
    '''
    # tokenize sentence to tokens (integers)
    idx = tokenizer.encode(txt)
    # convert to array of shape (1, num_words+2) - EOS and CLS added
    idx = np.array(idx)[None,:]
    # generate embeddings for each token - output is a tuple
    embedding = nlp(idx)
    # select first member of the tuple, remove first dimension which is 1 to get (num_words,embedding size 712)
    # exclude CLS and EOS tokens
    X = np.array(embedding[0][0][1:-1])
    return X

In [10]:
%%time
## create list of news vector
lst_mean_vecs = [utils_bert_embedding(txt, tokenizer, nlp).mean(0) for txt in tqdm(df["clean"])]

100%|██████████████████████████████████████| 4000/4000 [14:29<00:00,  4.60it/s]

Wall time: 14min 29s


In [11]:
X = np.array(lst_mean_vecs)
X.shape

(4000, 768)

In [12]:
np.savetxt("mean_vec.csv", X, delimiter="/")

In [13]:
print(X)

[[-2.5680837e-01  2.3240681e-01  1.6965662e-01 ... -8.8777855e-02
   1.4019369e-02  1.4557248e-01]
 [ 8.4562436e-02  3.2585591e-01  4.1670072e-01 ... -1.7362714e-01
   2.8379065e-01 -5.1412895e-02]
 [ 2.2545581e-01 -2.6025310e-01  3.2163188e-01 ... -3.3039427e-01
   1.9219832e-01  1.6776156e-01]
 ...
 [ 1.4312008e-01  1.5530565e-01  3.5850555e-01 ... -1.1386413e-01
   1.8839721e-01 -3.0466489e-04]
 [ 9.6061237e-02  8.8907346e-02  2.4147642e-01 ...  4.4398159e-02
   1.3453141e-01  1.7341268e-01]
 [ 8.6221755e-02 -2.6772487e-01  4.6793181e-01 ...  6.5415413e-03
  -4.6825301e-02  1.2560053e-01]]


In [16]:
from numpy import genfromtxt
my_data = genfromtxt('mean_vec.csv', delimiter='/')
print(my_data)

[[-2.56808370e-01  2.32406810e-01  1.69656619e-01 ... -8.87778550e-02
   1.40193691e-02  1.45572484e-01]
 [ 8.45624357e-02  3.25855911e-01  4.16700721e-01 ... -1.73627138e-01
   2.83790648e-01 -5.14128953e-02]
 [ 2.25455806e-01 -2.60253102e-01  3.21631879e-01 ... -3.30394268e-01
   1.92198321e-01  1.67761564e-01]
 ...
 [ 1.43120080e-01  1.55305654e-01  3.58505547e-01 ... -1.13864131e-01
   1.88397214e-01 -3.04664893e-04]
 [ 9.60612372e-02  8.89073461e-02  2.41476417e-01 ...  4.43981588e-02
   1.34531409e-01  1.73412681e-01]
 [ 8.62217546e-02 -2.67724872e-01  4.67931807e-01 ...  6.54154131e-03
  -4.68253009e-02  1.25600532e-01]]


In [17]:
dic_y = {k:utils_bert_embedding(v, tokenizer, nlp).mean(0) for k,v in tqdm(clusters.items())}

100%|████████████████████████████████████████████| 1/1 [00:00<00:00,  3.24it/s]


In [18]:
print(dic_y)

{'HATE': array([ 2.55093783e-01,  1.82706341e-01,  1.43618360e-01, -2.79080272e-01,
       -8.70466679e-02, -1.60564318e-01,  4.68662918e-01,  2.48518735e-01,
       -3.37478258e-02, -2.38933101e-01,  1.38003826e-01, -2.25163400e-01,
       -2.17556164e-01, -6.89167529e-02,  6.65362272e-03, -7.44596636e-03,
        4.67746705e-02,  1.38214305e-01, -2.00216159e-01,  5.14454246e-01,
        2.10825190e-01,  2.26417124e-01, -7.87845328e-02, -1.32102855e-02,
        2.26131126e-01,  3.41755569e-01,  4.84811276e-01, -2.55082369e-01,
       -2.25392237e-01, -1.87913895e-01,  5.69476008e-01,  1.76780358e-01,
        1.15006290e-01,  8.47342908e-02, -1.01412274e-01,  7.92896971e-02,
        3.70380022e-02,  2.15094939e-01,  1.86477810e-01,  2.30520338e-01,
       -3.88846040e-01, -3.03668469e-01,  4.04374078e-02,  3.10661614e-01,
       -3.50910813e-01, -3.87722999e-01, -3.09790015e-01,  1.61548346e-01,
        9.74037424e-02, -2.48916805e-01, -5.27213849e-02,  2.16238394e-01,
       -9.848772